# Tool Calling with Gemini

OpenAI's function calling was first deprecated in favor of tool calling - but now the correct term seems to be function calling again.

If you call it tool- or function calling - at the moment it does not really matter.

Reference: https://platform.openai.com/docs/guides/function-calling

In [14]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")
llm.invoke("How will the weather be in munich today?")

## Approach 1: Using @tool decorator (Recommended)

In [20]:
from langchain_core.tools import tool

@tool
def fake_weather_api(city: str) -> str:
    """
    Check the weather in a specified city.

    Args:
        city (str): The name of the city where you want to check the weather.

    Returns:
        str: A description of the current weather in the specified city.
    """
    return "Sunny, 22°C"

@tool
def outdoor_seating_availability(city: str) -> str:
    """
    Check if outdoor seating is available at a specified restaurant in a given city.

    Args:
        city (str): The name of the city where you want to check for outdoor seating availability.

    Returns:
        str: A message stating whether outdoor seating is available or not.
    """
    return "Outdoor seating is available."

tools = [fake_weather_api, outdoor_seating_availability]

## Approach 2: Using Pydantic classes

Note: If using this approach, you must also update the tool_mapping cell below to use the correct tool names.

In [21]:
# Alternative: Using Pydantic classes for tool definitions
# To use this approach:
# 1. Comment out the @tool decorator cell above
# 2. Uncomment this cell
# 3. Update the tool_mapping cell below to use correct names

from pydantic import BaseModel, Field

class fake_weather_api(BaseModel):
    """Check the weather in a specified city."""
    city: str = Field(..., description="Name of the city to check the weather for")

class outdoor_seating_availability(BaseModel):
    """Check if outdoor seating is available at a specified restaurant in a given city."""
    city: str = Field(..., description="Name of the city where the restaurant is located")

tools = [fake_weather_api, outdoor_seating_availability]

In [22]:
llm_with_tools = llm.bind_tools(tools)

In [23]:
result = llm_with_tools.invoke("How will the weather be in munich today?")
result

ChatGoogleGenerativeAIError: Error calling model 'gemini-2.5-flash-lite' (RESOURCE_EXHAUSTED): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash-lite\nPlease retry in 44.757771839s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerM

In [ ]:
result = llm_with_tools.invoke(
    "How will the weather be in munich today? Do you still have seats outdoor available?"
)
result

In [ ]:
result.tool_calls

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in munich today? I would like to eat outside if possible"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [ ]:
messages

In [ ]:
# Tool mapping for @tool decorator approach (default)
tool_mapping = {
    "fake_weather_api": fake_weather_api,
    "outdoor_seating_availability": outdoor_seating_availability,
}

# For Pydantic approach, use this mapping instead:
# tool_mapping = {
#     "fake_weather_api": fake_weather_api,
#     "outdoor_seating_availability": outdoor_seating_availability,
# }

In [ ]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [ ]:
messages

In [ ]:
llm_with_tools.invoke(messages)